In [1]:
# !pip install PyPDF2
# !pip install tika
# !pip install pysrt
# !pip install fuzzywuzzy
# !pip install spacy
# !pip install optuna

In [2]:
import pandas as pd
import numpy as np
import optuna
import nltk
import sklearn 
from sklearn.model_selection import train_test_split
import os
import PyPDF2
from tika import parser
import re 
import pysrt
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import warnings
from nltk import word_tokenize,pos_tag
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score
from sklearn.metrics import accuracy_score,recall_score, fbeta_score,roc_auc_score,plot_roc_curve
from sklearn.metrics import f1_score, precision_score,plot_confusion_matrix,classification_report
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from nltk import ngrams
import spacy
from spacy.lang.en import stop_words
from nltk.corpus import stopwords

C:\Users\new\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
warnings.filterwarnings('ignore')


In [4]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('tagsets')
# nltk.download('averaged_perceptron_tagger')

In [5]:
data = pd.read_csv(r'C:\Users\new\labels_all.csv')

In [6]:
data.describe()

,Movie,Kinopoisk,Level,Subtitles
count,116,69,116,88
unique,115,10,6,2
top,Sleepless in Seattle,Everything,B1,Yes
freq,2,19,38,85


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Movie       116 non-null    object
 1   Kinopoisk   69 non-null     object
 2   Level       116 non-null    object
 3   Subtitles   88 non-null     object
dtypes: object(4)
memory usage: 3.8+ KB


In [8]:
data.head()

,Movie,Kinopoisk,Level,Subtitles
0,Forrest Gump,Rus sub,"A2/A2+, B1",Yes
1,Finding Nemo\n,Everything,A2/A2+,Yes
2,Cast away\n,"Paid, Rus sub",A2/A2+,Yes
3,The invisible man (2020)\n,"Paid, Rus lan",A2/A2+,Yes
4,Back to the future\n,Rus sub,A2/A2+,Yes


In [9]:
data.columns

Index(['Movie', 'Kinopoisk ', 'Level', 'Subtitles'], dtype='object')

In [10]:
data.columns = [i.strip().lower() for i in data.columns]

In [11]:
data.columns

Index(['movie', 'kinopoisk', 'level', 'subtitles'], dtype='object')

In [12]:
data.drop(data.columns[1],axis=1,inplace=True)

In [13]:
films_wo_subtitles = np.array(data[data.subtitles.isna()]['movie'])

In [14]:
data.shape

(116, 3)

In [15]:
tmp = pd.DataFrame()

In [16]:
data['n_cat'] = data['level'].apply(lambda x: 1 if len(x) == 2 else 2 if len(x)==6 else 3)

In [17]:
data[data['n_cat'] == 3]

,movie,level,subtitles,n_cat
0,Forrest Gump,"A2/A2+, B1",Yes,3
12,The terminal\n,"A2/A2+, B1",Yes,3
28,Her,"A2/A2+, B1",Yes,3
78,Moulin Rouge 🎙️,"A2/A2+, B1",No,3
84,The fault in our stars 😭,"A2/A2+, B1",Yes,3


In [18]:
data['n_cat'].value_counts()

1    75
2    36
3     5
Name: n_cat, dtype: int64

In [19]:
data.head()

,movie,level,subtitles,n_cat
0,Forrest Gump,"A2/A2+, B1",Yes,3
1,Finding Nemo\n,A2/A2+,Yes,2
2,Cast away\n,A2/A2+,Yes,2
3,The invisible man (2020)\n,A2/A2+,Yes,2
4,Back to the future\n,A2/A2+,Yes,2


In [20]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F9DF"                 
                           "]+", flags=re.UNICODE)
def emoji_del(raw):
    return re.sub(emoji_pattern,'',raw)
# def brack_del(raw):
#     return re.sub(CLEANR_BRACK,'',raw)
CLEANR_INBRACK = re.compile(r'\(.*?\)()')
def del_in_brack(raw):
    return re.sub(CLEANR_INBRACK,'',raw)

data['movie'] = data['movie'].apply(emoji_del)
data['movie']  = data['movie'].apply(del_in_brack)


data.movie.unique()



array(['Forrest Gump', 'Finding Nemo\n', 'Cast away\n',
       'The invisible man \n', 'Back to the future\n', 'Twilight\n',
       'Toy story', 'The cabin in the woods\n', 'Up', 'Dredd\n',
       'Ready or not\n', 'The jungle book\n', 'The terminal\n',
       'The man called Flinstone', 'Batman begins',
       'Kubo and the two strings', 'Soul', 'The break-up', 'The holiday',
       'Sleepless in Seattle ', 'My big fat Greek wedding',
       'Groundhog day', 'The lion king', 'Home alone',
       'Catch me if you can', 'Mamma Mia', 'Harry Potter ', 'Matilda ',
       'Her', 'Shrek', 'Pirates of the Caribbean  ', 'The terminator',
       'Love actually ', 'The lord of the rings',
       'The theory of everything', 'We’re the Millers',
       'Meet the parents', 'Pulp Fiction', 'The graduate',
       'A star is born', 'The social network', 'The hangover',
       'Knives out', 'Bridget Jones diary', 'Die Hard',
       'The secret life of Walter Mitty', 'Mrs. Doubtfire',
       '10 things 

In [21]:
data['movie'] = data['movie'].apply(emoji_del)
data.movie.unique()

array(['Forrest Gump', 'Finding Nemo\n', 'Cast away\n',
       'The invisible man \n', 'Back to the future\n', 'Twilight\n',
       'Toy story', 'The cabin in the woods\n', 'Up', 'Dredd\n',
       'Ready or not\n', 'The jungle book\n', 'The terminal\n',
       'The man called Flinstone', 'Batman begins',
       'Kubo and the two strings', 'Soul', 'The break-up', 'The holiday',
       'Sleepless in Seattle ', 'My big fat Greek wedding',
       'Groundhog day', 'The lion king', 'Home alone',
       'Catch me if you can', 'Mamma Mia', 'Harry Potter ', 'Matilda ',
       'Her', 'Shrek', 'Pirates of the Caribbean  ', 'The terminator',
       'Love actually ', 'The lord of the rings',
       'The theory of everything', 'We’re the Millers',
       'Meet the parents', 'Pulp Fiction', 'The graduate',
       'A star is born', 'The social network', 'The hangover',
       'Knives out', 'Bridget Jones diary', 'Die Hard',
       'The secret life of Walter Mitty', 'Mrs. Doubtfire',
       '10 things 

In [22]:
data['movie'] = data['movie'].apply(lambda x: x.strip().lower())

In [23]:
data.head()

,movie,level,subtitles,n_cat
0,forrest gump,"A2/A2+, B1",Yes,3
1,finding nemo,A2/A2+,Yes,2
2,cast away,A2/A2+,Yes,2
3,the invisible man,A2/A2+,Yes,2
4,back to the future,A2/A2+,Yes,2


In [24]:
data['movie'] = data['movie'].apply(emoji_del)
data.movie.unique()

array(['forrest gump', 'finding nemo', 'cast away', 'the invisible man',
       'back to the future', 'twilight', 'toy story',
       'the cabin in the woods', 'up', 'dredd', 'ready or not',
       'the jungle book', 'the terminal', 'the man called flinstone',
       'batman begins', 'kubo and the two strings', 'soul',
       'the break-up', 'the holiday', 'sleepless in seattle',
       'my big fat greek wedding', 'groundhog day', 'the lion king',
       'home alone', 'catch me if you can', 'mamma mia', 'harry potter',
       'matilda', 'her', 'shrek', 'pirates of the caribbean',
       'the terminator', 'love actually', 'the lord of the rings',
       'the theory of everything', 'we’re the millers',
       'meet the parents', 'pulp fiction', 'the graduate',
       'a star is born', 'the social network', 'the hangover',
       'knives out', 'bridget jones diary', 'die hard',
       'the secret life of walter mitty', 'mrs. doubtfire',
       '10 things i hate about you', 'inside out', '

In [25]:
data['subtitles'] = np.where(data['subtitles']=='Yes',1,0)
data['subtitles'].unique()

array([1, 0])

In [26]:
data[data['movie'] == 'inside out']

,movie,level,subtitles,n_cat
48,inside out,B1,1,1
92,inside out,B1,0,1


In [27]:
data[data['n_cat'] == 3]

,movie,level,subtitles,n_cat
0,forrest gump,"A2/A2+, B1",1,3
12,the terminal,"A2/A2+, B1",1,3
28,her,"A2/A2+, B1",1,3
78,moulin rouge ️,"A2/A2+, B1",0,3
84,the fault in our stars,"A2/A2+, B1",1,3


In [28]:
# files = os.listdir(r'C:\Users\new\project_nlp\Subtitles_all')
# files = os.listdir(r'C:\Users\Egor\Subtitles_all')

In [29]:
print(fuzz.ratio('the fault in our stars 😭','the fault in our stars'))

96


In [30]:
data.shape

(116, 4)

In [31]:
data['movie'].duplicated().sum()

5

In [32]:
tmp_dup = data['movie'].duplicated().reset_index()

In [33]:
tmp_dup.columns = ['in','yes']

In [34]:
tmp_dup[tmp_dup['yes'] == 1]

,in,yes
88,88,True
92,92,True
94,94,True
95,95,True
104,104,True


In [35]:
data['movie'].duplicated().sum()

5

In [36]:
dup_indx = np.array(tmp_dup[tmp_dup['yes'] == 1]['in'])

In [37]:
dupl_name = np.array(data.iloc[dup_indx]['movie'])

In [38]:
dupl_movie_del = data.query('movie in @dupl_name').sort_values(by='movie')

In [39]:
row_to_del = np.array(dupl_movie_del[dupl_movie_del['subtitles'] == 0].index)

In [40]:
data.shape

(116, 4)

In [41]:
data.index

RangeIndex(start=0, stop=116, step=1)

In [42]:
row_to_del

array([ 92,  88,  94, 104,  95], dtype=int64)

In [43]:
data.drop(row_to_del,inplace=True)

In [44]:
data.shape

(111, 4)

In [45]:
# CLEANR_HTML = re.compile('<.*?>') 

CLEANR_HTML = re.compile(r'\<[^>]*\>')

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR_HTML, ' ', raw_html)
  return cleantext
CLEANR_BRACK = re.compile(r'[\([{})\]]')
# CLEANR_BRACK = re.compile(r'\(.*?\)()')
def cleanbrack(raw_brack):
  cleantext = re.sub(CLEANR_BRACK, ' ', raw_brack)
  return cleantext

In [46]:
cleanbrack('(fwfwqf)(fwqfq)')

' fwfwqf  fwqfq '

In [47]:
def rename_sub(new_lst,old_lst,chdir):
    os.chdir(chdir)
    directory = os.getcwd()
    for i in range(len(new_lst)):
        os.rename(os.path.join(directory,old_lst[i]),os.path.join(directory,new_lst[i]))

In [48]:
# def rename_sub(new_lst,old_lst,directory=os.getcwd()):
#     for i in range(len(new_lst)):
#         os.rename(os.path.join(directory,old_lst[i]),os.path.join(directory,new_lst[i]))

In [49]:
data.columns

Index(['movie', 'level', 'subtitles', 'n_cat'], dtype='object')

In [50]:
old_lst_files = os.listdir(r'C:\Users\new\Subtitles_all')

In [51]:
old_lst_files[:5]

['10 cloverfield lane.srt',
 '10 things i hate about you.srt',
 'a knights tale.srt',
 'a star is born.srt',
 'aladdin.srt']

In [52]:
new_lst_files = list(map(del_in_brack,old_lst_files))

In [53]:
new_lst_files = list(map(lambda x: x.replace('_',' '),new_lst_files))

In [54]:
new_lst_files = [x.lower() for x in new_lst_files]

In [55]:
new_lst_files

['10 cloverfield lane.srt',
 '10 things i hate about you.srt',
 'a knights tale.srt',
 'a star is born.srt',
 'aladdin.srt',
 'all dogs go to heaven.srt',
 'an american tail.srt',
 'babe.srt',
 'back to the future.srt',
 'banking on bitcoin.srt',
 'batman begins.srt',
 'beauty and the beast.srt',
 'before i go to sleep.srt',
 'before sunrise.srt',
 'before sunset.srt',
 'braveheart.srt',
 'breaking bad the movie.srt',
 'bridget jones diary.srt',
 'bridget joness baby.srt',
 'cars.srt',
 'casper.srt',
 'cast away.srt',
 'catboost info',
 'catch me if you can.srt',
 'charlie and the chocolate factory.srt',
 'cinderella.srt',
 'clueless.srt',
 'deadpool.srt',
 'despicable me.srt',
 'die hard.srt',
 'dredd.srt',
 'dune.srt',
 'enola holmes.srt',
 'entrapment.srt',
 'eurovision song contest .srt',
 'ferdinand.srt',
 'fight club.srt',
 'finding nemo.srt',
 'forrest gump.srt',
 'gogo loves english.srt',
 'good will hunting.srt',
 'groundhog day.srt',
 'harry potter and the philosophers stone.

In [56]:
for i in range(len(new_lst_files)):
    print(old_lst_files[i])
    print()
    print(new_lst_files[i])
    print('-'*40)

10 cloverfield lane.srt

10 cloverfield lane.srt
----------------------------------------
10 things i hate about you.srt

10 things i hate about you.srt
----------------------------------------
a knights tale.srt

a knights tale.srt
----------------------------------------
a star is born.srt

a star is born.srt
----------------------------------------
aladdin.srt

aladdin.srt
----------------------------------------
all dogs go to heaven.srt

all dogs go to heaven.srt
----------------------------------------
an american tail.srt

an american tail.srt
----------------------------------------
babe.srt

babe.srt
----------------------------------------
back to the future.srt

back to the future.srt
----------------------------------------
banking on bitcoin.srt

banking on bitcoin.srt
----------------------------------------
batman begins.srt

batman begins.srt
----------------------------------------
beauty and the beast.srt

beauty and the beast.srt
-------------------------------------

In [57]:
new_lst_files

['10 cloverfield lane.srt',
 '10 things i hate about you.srt',
 'a knights tale.srt',
 'a star is born.srt',
 'aladdin.srt',
 'all dogs go to heaven.srt',
 'an american tail.srt',
 'babe.srt',
 'back to the future.srt',
 'banking on bitcoin.srt',
 'batman begins.srt',
 'beauty and the beast.srt',
 'before i go to sleep.srt',
 'before sunrise.srt',
 'before sunset.srt',
 'braveheart.srt',
 'breaking bad the movie.srt',
 'bridget jones diary.srt',
 'bridget joness baby.srt',
 'cars.srt',
 'casper.srt',
 'cast away.srt',
 'catboost info',
 'catch me if you can.srt',
 'charlie and the chocolate factory.srt',
 'cinderella.srt',
 'clueless.srt',
 'deadpool.srt',
 'despicable me.srt',
 'die hard.srt',
 'dredd.srt',
 'dune.srt',
 'enola holmes.srt',
 'entrapment.srt',
 'eurovision song contest .srt',
 'ferdinand.srt',
 'fight club.srt',
 'finding nemo.srt',
 'forrest gump.srt',
 'gogo loves english.srt',
 'good will hunting.srt',
 'groundhog day.srt',
 'harry potter and the philosophers stone.

In [58]:
directory = os.getcwd()

In [59]:
old_lst_files

['10 cloverfield lane.srt',
 '10 things i hate about you.srt',
 'a knights tale.srt',
 'a star is born.srt',
 'aladdin.srt',
 'all dogs go to heaven.srt',
 'an american tail.srt',
 'babe.srt',
 'back to the future.srt',
 'banking on bitcoin.srt',
 'batman begins.srt',
 'beauty and the beast.srt',
 'before i go to sleep.srt',
 'before sunrise.srt',
 'before sunset.srt',
 'braveheart.srt',
 'breaking bad the movie.srt',
 'bridget jones diary.srt',
 'bridget joness baby.srt',
 'cars.srt',
 'casper.srt',
 'cast away.srt',
 'catboost_info',
 'catch me if you can.srt',
 'charlie and the chocolate factory.srt',
 'cinderella.srt',
 'clueless.srt',
 'deadpool.srt',
 'despicable me.srt',
 'die hard.srt',
 'dredd.srt',
 'dune.srt',
 'enola holmes.srt',
 'entrapment.srt',
 'eurovision song contest .srt',
 'ferdinand.srt',
 'fight club.srt',
 'finding nemo.srt',
 'forrest gump.srt',
 'gogo loves english.srt',
 'good will hunting.srt',
 'groundhog day.srt',
 'harry potter and the philosophers stone.

In [60]:

# os.chdir(r'C:\Users\new\Subtitles_all')
# directory = os.getcwd()
# for i in range(len(new_lst_files)):
#     os.rename(os.path.join(directory,old_lst_files[i]),os.path.join(directory,new_lst_files[i]))

In [61]:
data.level.value_counts()

B1            34
B2            28
A2/A2+        27
B1, B2         9
C1             8
A2/A2+, B1     5
Name: level, dtype: int64

In [62]:
after_rename_lst_sub = os.listdir(os.getcwd())



In [63]:
# def matchingr(row):
#     for i in range()
#     if fuzz.ratio(row,)

In [64]:
after_rename_lst_sub

['.anaconda',
 '.bash_history',
 '.conda',
 '.condarc',
 '.continuum',
 '.git',
 '.gitconfig',
 '.gnupg',
 '.ipynb_checkpoints',
 '.ipython',
 '.jupyter',
 '.lesshst',
 '.matplotlib',
 '.ms-ad',
 '.spyder-py3',
 'anaconda3',
 'AppData',
 'Application Data',
 'Contacts',
 'Cookies',
 'Desktop',
 'Documents',
 'Downloads',
 'English_level.zip',
 'Favorites',
 'first-project',
 'IntelGraphicsProfiles',
 'labels_all.csv',
 'Links',
 'Local Settings',
 'momo_q.csv',
 'Music',
 'NetHood',
 'nlp.ipynb',
 'NTUSER.DAT',
 'ntuser.dat.LOG1',
 'ntuser.dat.LOG2',
 'NTUSER.DAT{bad97b39-18d6-11ea-a20c-18cf5ec879d7}.TM.blf',
 'NTUSER.DAT{bad97b39-18d6-11ea-a20c-18cf5ec879d7}.TMContainer00000000000000000001.regtrans-ms',
 'NTUSER.DAT{bad97b39-18d6-11ea-a20c-18cf5ec879d7}.TMContainer00000000000000000002.regtrans-ms',
 'ntuser.ini',
 'Pictures',
 'PrintHood',
 'project_nlp',
 'PycharmProjects',
 'Recent',
 'Saved Games',
 'Sbornbli_project_2.ipynb',
 'Searches',
 'SendTo',
 'source',
 'Subtitles_all',
 '

In [65]:
data.head()

,movie,level,subtitles,n_cat
0,forrest gump,"A2/A2+, B1",1,3
1,finding nemo,A2/A2+,1,2
2,cast away,A2/A2+,1,2
3,the invisible man,A2/A2+,1,2
4,back to the future,A2/A2+,1,2


In [66]:
new_lst_files

['10 cloverfield lane.srt',
 '10 things i hate about you.srt',
 'a knights tale.srt',
 'a star is born.srt',
 'aladdin.srt',
 'all dogs go to heaven.srt',
 'an american tail.srt',
 'babe.srt',
 'back to the future.srt',
 'banking on bitcoin.srt',
 'batman begins.srt',
 'beauty and the beast.srt',
 'before i go to sleep.srt',
 'before sunrise.srt',
 'before sunset.srt',
 'braveheart.srt',
 'breaking bad the movie.srt',
 'bridget jones diary.srt',
 'bridget joness baby.srt',
 'cars.srt',
 'casper.srt',
 'cast away.srt',
 'catboost info',
 'catch me if you can.srt',
 'charlie and the chocolate factory.srt',
 'cinderella.srt',
 'clueless.srt',
 'deadpool.srt',
 'despicable me.srt',
 'die hard.srt',
 'dredd.srt',
 'dune.srt',
 'enola holmes.srt',
 'entrapment.srt',
 'eurovision song contest .srt',
 'ferdinand.srt',
 'fight club.srt',
 'finding nemo.srt',
 'forrest gump.srt',
 'gogo loves english.srt',
 'good will hunting.srt',
 'groundhog day.srt',
 'harry potter and the philosophers stone.

In [67]:
names_of_moviefile = list(map(lambda x: x.replace('.srt',''),new_lst_files))

In [68]:
movie_in_data = list(data['movie'])

In [69]:
len(names_of_moviefile)

115

In [70]:
len(movie_in_data)

111

In [71]:
# not_in_lst = []

# for i in range(len(names_of_moviefile)):
#     if names_of_moviefile[i] not in (movie_in_data):
#         not_in_lst.append(names_of_moviefile[i])

In [72]:
n_of_match = 0
tmp_lst_to_add = []
movie_name = []

In [73]:
for i in range(len(names_of_moviefile)):
    for j in range(len(movie_in_data)):
            if fuzz.ratio(names_of_moviefile[i],movie_in_data[j]) > 89:
                n_of_match += 1
                tmp_lst_to_add.append(names_of_moviefile[i]+'.srt')
                movie_name.append(movie_in_data[j])
                print(names_of_moviefile[i],movie_in_data[j])
                

10 cloverfield lane 10 cloverfield lane
10 things i hate about you 10 things i hate about you
a knights tale a knight’s tale
a star is born a star is born
aladdin aladdin
all dogs go to heaven all dogs go to heaven
an american tail an  american tail
babe babe
back to the future back to the future
banking on bitcoin banking on bitcoin
batman begins batman begins
beauty and the beast beauty and the beast
before i go to sleep before i go to sleep
before sunrise before sunrise
before sunset before sunset
braveheart braveheart
bridget jones diary bridget jones diary
bridget joness baby bridget jones’s baby
cars cars
cast away cast away
catch me if you can catch me if you can
charlie and the chocolate factory charlie and the chocolate factory
cinderella cinderella
clueless clueless
deadpool deadpool
despicable me despicable me
die hard die hard
dredd dredd
dune dune
enola holmes enola holmes
entrapment entrapment
ferdinand ferdinand
fight club fight club
finding nemo finding nemo
forrest gum

In [74]:
file_series = pd.DataFrame({'movie':movie_name,'subfile_name':tmp_lst_to_add})

In [75]:
file_series

,movie,subfile_name
0,10 cloverfield lane,10 cloverfield lane.srt
1,10 things i hate about you,10 things i hate about you.srt
2,a knight’s tale,a knights tale.srt
3,a star is born,a star is born.srt
4,aladdin,aladdin.srt
...,...,...
101,venom,venom.srt
102,warm bodies,warm bodies.srt
103,we’re the millers,we are the millers.srt
104,while you were sleeping,while you were sleeping.srt


In [76]:
data = pd.merge(data,file_series,how='left',on='movie')

In [77]:
data

,movie,level,subtitles,n_cat,subfile_name
0,forrest gump,"A2/A2+, B1",1,3,forrest gump.srt
1,finding nemo,A2/A2+,1,2,finding nemo.srt
2,cast away,A2/A2+,1,2,cast away.srt
3,the invisible man,A2/A2+,1,2,the invisible man.srt
4,back to the future,A2/A2+,1,2,back to the future.srt
...,...,...,...,...,...
106,klaus,C1,0,1,klaus.srt
107,ocean’s eleven,C1,0,1,oceans eleven.srt
108,ocean’s twelve,C1,0,1,oceans twelve.srt
109,bridget jones’s baby,C1,0,1,bridget joness baby.srt


In [78]:
data[data['subfile_name'].isna()]

,movie,level,subtitles,n_cat,subfile_name
26,harry potter,B1,1,1,NaN
58,lie to me,"B1, B2",0,2,NaN
81,eurovision song contest: the story of fire saga,A2/A2+,1,2,NaN
83,the walking dead,A2/A2+,0,2,NaN
102,the secret life of pets,B2,0,1,NaN


In [79]:
np.array(data[data['subfile_name'].isna()]['movie'])

array(['harry potter', 'lie to me',
       'eurovision song contest: the story of fire saga',
       'the walking dead', 'the secret life of pets'], dtype=object)

In [80]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [81]:
os.getcwd()


'C:\\Users\\new'

In [82]:
data.iloc[13]

movie                the man called flinstone
level                                  A2/A2+
subtitles                                   1
n_cat                                       2
subfile_name    the man called flintstone.srt
Name: 13, dtype: object

In [83]:
data.head()

,movie,level,subtitles,n_cat,subfile_name
0,forrest gump,"A2/A2+, B1",1,3,forrest gump.srt
1,finding nemo,A2/A2+,1,2,finding nemo.srt
2,cast away,A2/A2+,1,2,cast away.srt
3,the invisible man,A2/A2+,1,2,the invisible man.srt
4,back to the future,A2/A2+,1,2,back to the future.srt


In [84]:
def sub_reader_cleaner(data_,
                       movie_column,
                       file_column,
                       subtitles_folder,
                       lst_column_=True,
                       set_column_=True,
                       tokekise_column=True,
                       lemma_column=True,
                      cwd_to_fold=False):
    global path_to_subs
    tmp_array= np.array(data_[file_column])
        
    if cwd_to_fold == False:
        os.chdir(os.path.join(os.getcwd(),subtitles_folder))
        path_to_subs = os.getcwd()
    else:
        path_to_subs = os.path.join(os.getcwd())
        
    for i in range(len(tmp_array)):
        lst_column = []
        subs = pysrt.open(os.path.join(path_to_subs,tmp_array[i]),encoding='latin-1')
        clean_text= cleanbrack(cleanhtml(subs.text).strip(',.<>/?!-')).lower()
        lst_column.append(clean_text)
        if lst_column_:
            data_.loc[i,'sub_list'] = lst_column
#         if set_column_:
#             data_.loc[i,'sub_set'] = clean_text.split(' ')

    


    

In [85]:
data_wo_na = data.dropna()
data_wo_na.reset_index(inplace=True,drop=True)





In [86]:
data_wo_na

,movie,level,subtitles,n_cat,subfile_name
0,forrest gump,"A2/A2+, B1",1,3,forrest gump.srt
1,finding nemo,A2/A2+,1,2,finding nemo.srt
2,cast away,A2/A2+,1,2,cast away.srt
3,the invisible man,A2/A2+,1,2,the invisible man.srt
4,back to the future,A2/A2+,1,2,back to the future.srt
...,...,...,...,...,...
101,klaus,C1,0,1,klaus.srt
102,ocean’s eleven,C1,0,1,oceans eleven.srt
103,ocean’s twelve,C1,0,1,oceans twelve.srt
104,bridget jones’s baby,C1,0,1,bridget joness baby.srt


In [87]:
sub_reader_cleaner(data_wo_na,'movie','subfile_name','Subtitles_all',cwd_to_fold=False)

In [88]:
def list_n_to_list(row):
    return ' '.join(row).split('\n')

In [89]:
data_wo_na['sub_list'] = data_wo_na['sub_list'].apply(list_n_to_list)

In [90]:
def strip_punkt(row_punkt):
    return list(map(lambda x: x.strip(',.<>/?\\!@#$%^&*\'()_++?:%:;№!",{}[]-'),row_punkt))

In [91]:
data_wo_na['sub_list'] = data_wo_na['sub_list'].apply(strip_punkt)

In [92]:
data_wo_na.head()

,movie,level,subtitles,n_cat,subfile_name,sub_list
0,forrest gump,"A2/A2+, B1",1,3,forrest gump.srt,"[hello, my name's forrest. forrest gump, do yo..."
1,finding nemo,A2/A2+,1,2,finding nemo.srt,"[ music playing , advertise your product or br..."
2,cast away,A2/A2+,1,2,cast away.srt,[created and encoded by -- bokutox -- of www...
3,the invisible man,A2/A2+,1,2,the invisible man.srt,"[adrian, come on, zeus, i'm sorry, i can't ta..."
4,back to the future,A2/A2+,1,2,back to the future.srt,"[october is inventory time, so right now, stat..."


In [93]:
dct = {'A1':'0','A2+':'1.5','A2':'1','B1':'2','B2':'3','C1':'4','C2':'5'}

In [94]:


CLEANR_SLASH = re.compile(r'/|,')
def slash_dot(raw_lash):
    return re.sub(CLEANR_SLASH, ' ', raw_lash)


In [95]:
data_wo_na['level'] = data_wo_na['level'].apply(slash_dot)

In [96]:
data_wo_na['level'].unique()

array(['A2 A2+  B1', 'A2 A2+', 'B1', 'B1  B2', 'B2', 'C1'], dtype=object)

In [97]:
data_wo_na.level.unique()

array(['A2 A2+  B1', 'A2 A2+', 'B1', 'B1  B2', 'B2', 'C1'], dtype=object)

In [98]:
# def level_numerise(row):
#     for i in range(len(row.split())):
#         for key,values in dct.items():
#             if key in row:
#                 row = row.replace(key,values)
#             return row

In [99]:
for i in range(len(data_wo_na.level)):
    for key,values in dct.items():
        if key in data_wo_na.level[i]:
            data_wo_na.level[i] = data_wo_na.level[i].replace(key,values)
    print(data_wo_na.level[i])

1 1.5  2
1 1.5
1 1.5
1 1.5
1 1.5
1 1.5
1 1.5
1 1.5
1 1.5
1 1.5
1 1.5
1 1.5
1 1.5  2
1 1.5
1 1.5
1 1.5
2
1 1.5
2
1 1.5
1 1.5
2
1 1.5
2
2  3
2
2
1 1.5  2
2
2
2
2
2  3
3
2
2
3
2  3
3
3
3
3
2  3
2
2
2
2
2
3
2
3
3
3
3
3
3
2  3
2
2  3
2
3
1 1.5
2
3
3
1 1.5
1 1.5
2
2
2
1 1.5
1 1.5
3
2  3
2
3
1 1.5  2
1 1.5
1 1.5
2
1 1.5  2
2
2  3
3
2
2
2
2
2
2
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4


In [100]:
# data_wo_na['level'].apply(level_numerise)

In [101]:
data_wo_na['level'] = data_wo_na['level'].apply(lambda x: x.split())

In [102]:
def split_n_numeric(row):
    return list(map(lambda x: float(x),row))

In [103]:
data_wo_na['level'].apply(split_n_numeric)

0      [1.0, 1.5, 2.0]
1           [1.0, 1.5]
2           [1.0, 1.5]
3           [1.0, 1.5]
4           [1.0, 1.5]
            ...       
101              [4.0]
102              [4.0]
103              [4.0]
104              [4.0]
105              [4.0]
Name: level, Length: 106, dtype: object

In [104]:
def to_set(row):
    return set(row)

In [105]:
# MD  Modal verb (can, could, may, must)
# VB  Base verb (take)
# VBC Future tense, conditional
# VBD Past tense (took)
# VBF Future tense
# VBG Gerund, present participle (taking)
# VBN Past participle (taken)
# VBP Present tense (take)
# VBZ Present 3rd person singular (takes)

In [106]:
data_wo_na['sub_set'] = data_wo_na['sub_list'].apply(to_set)

In [107]:
data_wo_na['sub_big_string'] = data_wo_na['sub_list'].apply(lambda x: ' '.join(x))

In [108]:
    data_wo_na['sent_tokenise'] = data_wo_na['sub_big_string'].apply(sent_tokenize)

In [109]:
text = word_tokenize(data_wo_na['sub_big_string'][0])
tagged = pos_tag(text)

In [110]:
conjunction_lst =     ['CC']
cardinal_lst =['CD']
determiner_lst =['DT','PDT','WDT']
existential_lst = ['EX']
foreign_word_lst = ['FW']
preposition_lst =['IN']
modal_lst =['MD']
adjective_lst =['JJ','JJR','JJS']
noun_lst =['NNP','NNPS','NN','NNS'] 
genitive_lst =['POS']
pronoun_lst =['PRP','PRP$','WP','WP$']
adverb_lst =['RB','RBR','RBS']
particle_lst =['RP']
to_lst =['TO']
interjection_lst =['UH']
verb_lst =['VB','VBD','VBG','VBN','VBP','VBZ']
    


In [111]:
data_wo_na['conjunction'] = 0
data_wo_na['cardinal'] = 0
data_wo_na['determiner'] = 0
data_wo_na['existential'] = 0
data_wo_na['foreign_word'] = 0
data_wo_na['preposition'] = 0
data_wo_na['modal'] = 0
data_wo_na['adjective'] = 0
data_wo_na['noun'] = 0
data_wo_na['genitive'] = 0
data_wo_na['pronoun'] = 0
data_wo_na['adverb'] = 0
data_wo_na['particle'] = 0
data_wo_na['to'] = 0
data_wo_na['interjection'] = 0
data_wo_na['verb'] = 0

In [112]:
#     data_['CC'] = 
#     data_['CD'] = 
#     data_['DT','PDT','WDT'] = 
#     data_['EX'] = 
#     data_['FW'] = 
#     data_['IN'] = 
#     data_['MD'] = 
#     data_['JJ','JJR','JJS'] = 
#     data_['NNP','NNPS','NN','NNS'] = 
#     data_['POS'] = 
#     data_['PRP','PRP$','WP','WP$'] = 
#     data_['RB','RBR','RBS'] = 
#     data_['RP'] = 
#     data_['TO'] = 
#     data_['UH'] = 
#     data_['VB','VBD','VBG','VBN','VBP','VBZ'] = 

In [113]:
for i in range(len(data_wo_na['sub_big_string'])):
    text =word_tokenize(data_wo_na['sub_big_string'][i])
    tagged = pos_tag(text)
    for word, tag in tagged:
        if tag in conjunction_lst:
            data.loc[i,'conjunction'] += 1
        if tag in cardinal_lst:
            data.loc[i,'cardinal'] += 1
        if tag in determiner_lst:
            data.loc[i,'determiner'] += 1
        if tag in existential_lst: 
            data.loc[i,'existential'] += 1
        if tag in foreign_word_lst:
            data.loc[i,'foreign_word'] += 1
        if tag in preposition_lst:
            data.loc[i,'preposition'] += 1
        if tag in modal_lst:
            data.loc[i,'modal'] += 1
        if tag in adjective_lst:
            data.loc[i,'adjective'] += 1
        if tag in noun_lst:
            data.loc[i,'noun'] += 1
        if tag in genitive_lst:
            data.loc[i,'genitive'] += 1
        if tag in pronoun_lst:
            data.loc[i,'pronoun'] += 1
        if tag in adverb_lst:
            data.loc[i,'adverb'] += 1
        if tag in particle_lst:
            data.loc[i,'particle'] += 1
        if tag in to_lst:
            data.loc[i,'to'] += 1
        if tag in interjection_lst:
            data.loc[i,'interjection'] += 1
        if tag in verb_lst:
            data.loc[i,'verb'] += 1
        

In [114]:
data_wo_na['level'] = data_wo_na['level'].apply(lambda x: list(map(lambda x: float(x.strip()),x)))

In [115]:
data_wo_na['mean_level'] = data_wo_na['level'].apply(lambda x: sum(x)/len(x))

In [116]:
data_wo_na.columns

Index(['movie', 'level', 'subtitles', 'n_cat', 'subfile_name', 'sub_list',
       'sub_set', 'sub_big_string', 'sent_tokenise', 'conjunction', 'cardinal',
       'determiner', 'existential', 'foreign_word', 'preposition', 'modal',
       'adjective', 'noun', 'genitive', 'pronoun', 'adverb', 'particle', 'to',
       'interjection', 'verb', 'mean_level'],
      dtype='object')

In [318]:
features = data_wo_na.drop(['mean_level','mean_level_class','word_tokenise'],axis=1).loc[:,'conjunction':'stopwords_count']
target_numeric = data_wo_na['mean_level']

In [355]:
features_train,features_test,target_numeric_train,target_numeric_test = train_test_split(features,target_numeric,stratify=target_numeric,test_size=0.5,random_state=7)

In [356]:
lr = LinearRegression()
lr.fit(features_train,target_numeric_train)def
predict = lr.predict(features_test)


In [357]:
mean_absolute_error(target_numeric_test,predict)

0.713146707571551

In [358]:
mean_squared_error(target_numeric_test,predict)

0.7354426107702662

In [359]:
r2_score(target_numeric_test,predict)

-0.09798474284011571

In [360]:
# LogReg

In [125]:
data_wo_na['mean_level_class'] = data_wo_na['mean_level'].apply(lambda x: str(x))

In [126]:
def gramm_2(row):
    n_2=2
    gramms_2 = list(ngrams(row.split(),n_2))
    return gramms_2

In [127]:
# data_wo_na['gramm_2']= data_wo_na['sub_big_string'].apply(gramm_2)

In [128]:
def gramm_3(row):
    n_3=3

    gramms_3 = list(ngrams(row.split(),n_3))

    return gramms_3

In [129]:
# data_wo_na['gramm_3'] = data_wo_na['sub_big_string'].apply(gramm_3)

In [254]:
# !pip install streamlit

  Using cached streamlit-1.17.0-py2.py3-none-any.whl (9.3 MB)
  Using cached rich-13.3.1-py3-none-any.whl (239 kB)
  Using cached blinker-1.5-py2.py3-none-any.whl (12 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.


  Using cached Pympler-1.0.1-py3-none-any.whl (164 kB)
  Using cached pydeck-0.8.0-py2.py3-none-any.whl (4.7 MB)
  Using cached GitPython-3.1.30-py3-none-any.whl (184 kB)
  Using cached validators-0.20.0-py3-none-any.whl
  Using cached altair-4.2.2-py3-none-any.whl (813 kB)
  Using cached tzlocal-4.2-py3-none-any.whl (19 kB)
  Using cached semver-2.13.0-py2.py3-none-any.whl (12 kB)
  Using cached pyarrow-11.0.0-cp39-cp39-win_amd64.whl (20.6 MB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Using cached markdown_it_py-2.1.0-py3-none-any.whl (84 kB)
  Using cached Pygments-2.14.0-py3-none-any.whl (1.1 MB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
  Using cached pytz_deprecation_shim-0.1.0.post0-py2.py3-none-any.whl (15 kB)
  Using cached tzdata-2022.7-py2.py3-none-any.whl (340 kB)
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.11.2
    Uninstalling Pygments-2.11.2:
      Successfully un

In [184]:
def vectorise_2(row):
    vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2),lowercase=False)
    x2 = vectorizer2.fit_transform(row)
    return x2.toarray()

In [188]:
# data_wo_na['vectorise_2'] = data_wo_na['sent_tokenise'].apply(vectorise_2)

In [132]:
# !pip install catboost

In [171]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=149,
    random_seed=7,
    loss_function='MultiClass'
)

model.fit(
    features_train, target_train,
    eval_set=(features_test, target_test),
    verbose=False
)


In [172]:
accuracy_score(target_test,model.predict(features_test))

0.45454545454545453

In [135]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,cardinal,10.434424
1,conjunction,10.415676
2,interjection,9.281841
3,foreign_word,8.673547
4,existential,7.852793
5,preposition,7.678842
6,verb,6.366528
7,genitive,6.213563
8,modal,5.673620
9,particle,4.998099


In [136]:
# data_wo_na['vectorised_bigrams'] = data_wo_na['sub_list'].apply(vectorise_2)

In [137]:
def vectorise_3(row):
    vectorizer3 = CountVectorizer(analyzer='word', ngram_range=(3, 3),lowercase=False)
    x3 = vectorizer3.fit_transform(row)
    return x3.toarray()

In [138]:
# data_wo_na['vectorised_trgrams'] = data_wo_na['sub_list'].apply(vectorise_3)

In [139]:
sw_lst = stopwords.words('english')



def stopw_count(row):
    stop_c = 0
    for i in range(len(row)):
        if row[i] in sw_lst:
            stop_c+=1
    return stop_c



In [407]:
dct = {'A1':'0','A2+':'1.5','A2':'1','B1':'2','B2':'3','C1':'4','C2':'5'}
my_dict2 = {y: x for x, y in dct.items()}

In [408]:
my_dict2

{'0': 'A1',
 '1.5': 'A2+',
 '1': 'A2',
 '2': 'B1',
 '3': 'B2',
 '4': 'C1',
 '5': 'C2'}

In [140]:
data_wo_na['word_tokenise'] = data_wo_na['sub_big_string'].apply(word_tokenize)

In [141]:
data_wo_na['stopwords_count'] = data_wo_na['word_tokenise'].apply(stopw_count)



In [380]:
features = data_wo_na.drop(['mean_level','mean_level_class','word_tokenise'],axis=1).loc[:,'conjunction':'stopwords_count']
target = data_wo_na['mean_level_class']

In [381]:
features_train,features_test,target_train,target_test = train_test_split(features,target,test_size=0.7,stratify=target,random_state=7)

In [374]:
features_valid,features_test,target_valid,target_test = train_test_split(features_tv,target_tv,test_size=0.5,stratify=target_tv,random_state=7)

In [382]:
rs=7
#  с помощью библиотеки optuna 
#  подберем лучшие гиперпараметры
def objective(trial):
    clf_name = trial.suggest_categorical('classifier',['RandomForest'])
    if clf_name == 'RandomForest':
        max_depth = trial.suggest_int('max_depth',5,15)
        n_estimators = trial.suggest_int('n_estimators',5,100)
        criterion = trial.suggest_categorical('criterion',['gini','entropy'])
        clf_obj = RandomForestClassifier(criterion=criterion,max_depth=max_depth,n_estimators=n_estimators,random_state=rs)
    clf_obj.fit(features_train,target_train)
    score = f1_score(target_test,clf_obj.predict(features_test),average='weighted')
    accuracy_score_ = score.mean()
    return accuracy_score_



In [383]:
np.random.seed(7)
if __name__ == '__main__':
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials =200)
    print(study.best_trial)

[I 2023-02-10 17:09:03,473] A new study created in memory with name: no-name-e7065e1c-214f-4e90-a1ef-1f3810d5cfcb
[I 2023-02-10 17:09:03,573] Trial 0 finished with value: 0.3489584705544578 and parameters: {'classifier': 'RandomForest', 'max_depth': 15, 'n_estimators': 37, 'criterion': 'entropy'}. Best is trial 0 with value: 0.3489584705544578.
[I 2023-02-10 17:09:03,636] Trial 1 finished with value: 0.29255242557568134 and parameters: {'classifier': 'RandomForest', 'max_depth': 15, 'n_estimators': 19, 'criterion': 'entropy'}. Best is trial 0 with value: 0.3489584705544578.
[I 2023-02-10 17:09:03,689] Trial 2 finished with value: 0.3375858250276855 and parameters: {'classifier': 'RandomForest', 'max_depth': 10, 'n_estimators': 19, 'criterion': 'gini'}. Best is trial 0 with value: 0.3489584705544578.
[I 2023-02-10 17:09:03,859] Trial 3 finished with value: 0.39616706067769897 and parameters: {'classifier': 'RandomForest', 'max_depth': 13, 'n_estimators': 72, 'criterion': 'gini'}. Best i

[I 2023-02-10 17:09:08,972] Trial 33 finished with value: 0.3847374847374847 and parameters: {'classifier': 'RandomForest', 'max_depth': 15, 'n_estimators': 81, 'criterion': 'gini'}. Best is trial 18 with value: 0.4060703637447824.
[I 2023-02-10 17:09:09,204] Trial 34 finished with value: 0.3841555607513054 and parameters: {'classifier': 'RandomForest', 'max_depth': 9, 'n_estimators': 88, 'criterion': 'gini'}. Best is trial 18 with value: 0.4060703637447824.
[I 2023-02-10 17:09:09,335] Trial 35 finished with value: 0.3816356589147287 and parameters: {'classifier': 'RandomForest', 'max_depth': 11, 'n_estimators': 62, 'criterion': 'gini'}. Best is trial 18 with value: 0.4060703637447824.
[I 2023-02-10 17:09:09,451] Trial 36 finished with value: 0.33513019280461137 and parameters: {'classifier': 'RandomForest', 'max_depth': 12, 'n_estimators': 39, 'criterion': 'entropy'}. Best is trial 18 with value: 0.4060703637447824.
[I 2023-02-10 17:09:09,636] Trial 37 finished with value: 0.397227513

KeyboardInterrupt: 

In [ ]:
valid train params={'classifier': 'RandomForest', 'max_depth': 11, 'n_estimators': 15, 'criterion': 'gini'}

In [ ]:
features_tv
target_tv

In [221]:
params={'classifier': 'RandomForest', 'max_depth': 12, 'n_estimators': 20, 'criterion': 'gini'}

In [ ]:
params={'classifier': 'RandomForest', 'max_depth': 8, 'n_estimators': 22, 'criterion': 'entropy'}

In [ ]:
params={'classifier': 'RandomForest', 'max_depth': 8, 'n_estimators': 58, 'criterion': 'entropy'}

In [ ]:
params={'classifier': 'RandomForest', 'max_depth': 10, 'n_estimators': 53, 'criterion': 'entropy'}

In [396]:
model_rfc(10,53,'entropy')

Точность равна: 0.41333333333333333
F1 равна: 0.3841587301587302


In [397]:
def model_rfc(max_depth,n_estimators,criterion):
    RFC = RandomForestClassifier(max_depth=max_depth,n_estimators=n_estimators,criterion=criterion)
    RFC.fit(features_train,target_train)
    pr = RFC.predict(features_test)
    print(f'Точность равна: {accuracy_score(target_test,pr)}')
    print(f"F1 равна: {f1_score(target_test,pr,average='weighted')}")

In [398]:
def model_rfc(max_depth,n_estimators,criterion):
    RFC = RandomForestClassifier(max_depth=max_depth,n_estimators=n_estimators,criterion=criterion)
    RFC.fit(features_train,target_train)
    pr = RFC.predict(features_test)
    print(f'Точность равна: {accuracy_score(target_test,pr)}')
    print(f"F1 равна: {f1_score(target_test,pr,average='weighted')}")

In [399]:
model_rfc(5,75,'gini')

Точность равна: 0.36
F1 равна: 0.34983386022451446


In [400]:
model_rfc(8,22,'entropy')

Точность равна: 0.29333333333333333
F1 равна: 0.267817032109715


In [401]:
model_rfc(10,57,'entropy')

Точность равна: 0.4266666666666667
F1 равна: 0.4140873829595634


In [402]:
model_rfc(8,56,'entropy')

Точность равна: 0.32
F1 равна: 0.3058916659626765


In [403]:
model_rfc(13,59,'entropy')

Точность равна: 0.41333333333333333
F1 равна: 0.38619832255410563


In [404]:
model_rfc(11,74,'entropy')

Точность равна: 0.4
F1 равна: 0.3817616262879422


In [405]:
model_rfc(12,20,'gini')

Точность равна: 0.36
F1 равна: 0.3219194139194139


In [ ]:
plot_confusion_matrix(RFC,features_test,target_test)






In [ ]:
# # для тестовой и валидационной выборках лучшей модели
# def metrics_check(target_test,predict_test):
#     print(f'Accuracy для валидационной выборки: {accuracy_score(target_test,predict_test)}')
#     print('-'*100)
#     print(f'F1-score для валидационной выборки: {f1_score(target_test,predict_test)}')
#     print('-'*100)
#     print(f'Precision score для валидационной выборки: {precision_score(target_test,predict_test)}')
#     print('-'*100)
#     print(f'Recall score для валидационной выборки: {recall_score(target_test,predict_test)}')
#     print('-'*100)
#     print(f'F-beta score для валидационной выборки: {fbeta_score(target_test,predict_test,beta=1)}')
#     print('-'*100)

In [235]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(features_train, target_train)
accuracy_test = accuracy_score(target_test, model.predict(features_test))
accuracy_valid =  accuracy_score(target_valid, model.predict(features_valid))

In [387]:
accuracy_test

0.30666666666666664

In [388]:
accuracy_valid

0.3125

In [238]:
from sklearn.naive_bayes import BernoulliNB

In [384]:
model = BernoulliNB()
model.fit(features_train, target_train)
accuracy_test = accuracy_score(target_test, model.predict(features_test))
accuracy_valid =  accuracy_score(target_valid, model.predict(features_valid))

In [385]:
accuracy_test

0.30666666666666664

In [386]:
accuracy_valid

0.3125

In [242]:
from sklearn.ensemble import ExtraTreesClassifier

In [243]:
model = ExtraTreesClassifier()
model.fit(features_train, target_train)
accuracy_test = accuracy_score(target_test, model.predict(features_test))
accuracy_valid =  accuracy_score(target_valid, model.predict(features_valid))

In [244]:
accuracy_test

0.4090909090909091

In [245]:
accuracy_valid

0.5238095238095238

In [ ]:
print(classification_report(target_test,RFC.predict(features_test)))

In [ ]:

metrics_check(target_test,RFC.predict(features_test))

In [ ]:
RFC.predict(features_test)

In [ ]:
tags_lst = ['conjunction', 'cardinal',
       'determiner', 'existential', 'foreign_word', 'preposition', 'modal',
       'adjective', 'noun', 'genitive', 'pronoun', 'adverb', 'particle', 'to',
       'interjection', 'verb']

In [ ]:
sw_lst = stopwords.words('english')

sw_lst

def stopw_count(row):
    stop_c = 0
    for i in range(len(row)):
        if row[i] in sw_lst:
            stop_c+=1
    return stop_c

data_wo_na['stopwords_count'] = data_wo_na['word_tokenise'].apply(stopw_count)

data_wo_na['word_tokenise'] = data_wo_na['sub_big_string'].apply(word_tokenize)

In [ ]:
# nltk.help.upenn_tagset()


In [ ]:
# def tagger(row):
#     conjunction = 0
#     cardinal =0
#     determiner =0
#     existential = 0
#     foreign_word = 0
#     preposition =0
#     modal =0
#     adjective =0
#     noun =0
#     genitive =0
#     pronoun =0
#     adverb =0
#     particle =0
#     to =0
#     interjection =0
#     verb =0
#     text =word_tokenize(row['sub_big_string'])
#     tagged = pos_tag(text)
#     for word, tag in tagged:
#         if tag in conjunction_lst:
#             row['conjunction'] += 1
#         if tag in cardinal_lst:
#             row['cardinal'] += 1
#         if tag in determiner_lst:
#             row['determiner'] += 1
#         if tag in existential_lst: 
#             row['existential'] += 1
#         if tag in foreign_word_lst:
#             row['foreign_word'] += 1
#         if tag in preposition_lst:
#             row['preposition'] += 1
#         if tag in modal_lst:
#             row['modal'] += 1
#         if tag in adjective_lst:
#             row['adjective'] += 1
#         if tag in noun_lst:
#             row['noun'] += 1
#         if tag in genitive_lst:
#             row['genitive'] += 1
#         if tag in pronoun_lst:
#             row['pronoun'] += 1
#         if tag in adverb_lst:
#             row['adverb'] += 1
#         if tag in particle_lst:
#             row['particle'] += 1
#         if tag in to_lst:
#             row['to'] += 1
#         if tag in interjection_lst:
#             row['interjection'] += 1
#         if tag in verb_lst:
#             row['verb'] += 1
  


In [ ]:
def to_numeric(row_list):
    return list(map(,row))

In [ ]:
# data_wo_na['level'].apply(to_numeric)

In [ ]:
# def tags(row):
#     global tags_lst
#     tags_lst = []
#     tags_lst.append(pos_tag(row.split(' ')))

In [ ]:
# stopwords.words('english')

In [ ]:
# data_wo_na['sent_tokenise'].apply(len)

In [ ]:
# data_wo_na.head()

In [ ]:
# subs = pysrt.open(os.path.join(path_to_subs,'forrest gump.srt'),encoding='latin-1')
# cleanbrack(cleanhtml(subs.text).strip(',.<>/?!-')).lower().split('\n')
# # .replace('\n','')

In [ ]:
# for i in pysrt.open(r"C:\Users\new\Subtitles_all\10 cloverfield lane.srt"):
#     print(cleanhtml(i))

In [ ]:
#     if tokenise_column:
#         data['tokenise_lst'] = 
        
#     if lemma_column:
#         data['lemma_lst'] = 
#     if lemma_column and tokenise_column:
#         data['tokenise_lemma_lst'] = 